### Librabries Imported

In [96]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Extraction

####### Query Used
select io.id as invoice_id,od.billing_unit_count as unit_count,io.created_date as invoice_date,od.rate as unit_price,eu.id as customer_id,eu.client_type as customer_type,p.name as partner
from orders od
join invoice io on io.id=od.invoice_id and io.active=true
join enquiry e on od.enquiry_id = e.id and e.active=true
join eos_user eu on e.eos_user_id = eu.id and eu.active=true
join partner p on eu.partner_id = p.id and p.active=true
join file f on f.enquiry_id=e.id and f.active=true
where od.active=true
and (io.created_date >= '2019-08-01 00:00:00' and io.created_date <= '2019-12-31 23:59:59')
group by io.id,io.created_date,od.rate,eu.id,p.name,od.created_date,od.billing_unit_count
order by od.created_date desc;

In [97]:
DataFrame= pd.read_csv("C:\\Program Files\\Python36\\suven\\Notebooks\\customer_segmentation.csv",encoding='unicode_escape')
DataFrame.head()

,invoice_id,unit_count,invoice_date,unit_price,customer_id,customer_type,partner,total_price
0,1040364,362,2019-12-31 15:42:38,70.000,10964,individual,KOR,25340.00
1,1040544,200,2019-12-31 22:53:59,6.000,259217,individual,JPN,1200.00
2,1040289,749,2019-12-31 14:02:56,120.000,183017,individual,KOR,89880.00
3,1040925,214,2019-12-31 00:00:00,0.245,287569,corporate,CHN,52.43
4,1040908,3385,2019-12-31 00:00:00,0.235,276383,corporate,CHN,795.47


In [98]:
DataFrame.shape

(89125, 8)

#### Customer Distribution partner wise 

In [99]:
#DataFrame=DataFrame[DataFrame['customer_type']=='individual']
cust_data_partner=DataFrame[['customer_id','partner']].drop_duplicates()
cust_data_partner.groupby(['partner'])['customer_id'].aggregate('count').reset_index().sort_values('customer_id',ascending=False)

,partner,customer_id
7,JPN,18372
9,KOR,7377
1,CHN,6761
2,ENG,2860
0,BRL,1510
17,TWN,1140
16,TFG,974
19,WASG,749
20,WKH,573
14,SAGEG,286


#### Keeping Highest performing partner for both retail and corporate 

In [100]:
DataFrame=DataFrame.query("partner=='JPN'").reset_index(drop=True)

#### Check Missing Values in Dataset/DataFrame  

In [101]:
DataFrame.isnull().sum(axis=0)

invoice_id       0
unit_count       0
invoice_date     0
unit_price       0
customer_id      0
customer_type    0
partner          0
total_price      0
dtype: int64

#### Validate the negative value in unit price column 

In [102]:
DataFrame.unit_count.min()

1

#### Removing Data with unit_price as 0 because it can be a test inq   

In [103]:
DataFrame=DataFrame[(DataFrame['unit_count']>0)]

In [104]:
DataFrame.head()

,invoice_id,unit_count,invoice_date,unit_price,customer_id,customer_type,partner,total_price
0,1040544,200,2019-12-31 22:53:59,6.0,259217,individual,JPN,1200.0
1,1040437,139,2019-12-31 17:12:36,18.0,112272,individual,JPN,2502.0
2,1040081,364,2019-12-31 10:58:57,12.0,9551,individual,JPN,4368.0
3,1039912,1,2019-12-30 23:37:54,1.0,596720,corporate,JPN,1.0
4,1040222,998,2019-12-31 12:44:04,6.0,172039,individual,JPN,5988.0


### RFM Modelling Implementation

####R=Latest Date-Latest invoice date,F=count of invoice no. of transaction(s),M=sum of Total Amount of each client

In [105]:
import datetime as dt

In [106]:
DataFrame.invoice_date=DataFrame.invoice_date.str[0:10]
DataFrame['invoice_date'] = pd.to_datetime(DataFrame['invoice_date'])
DataFrame.head()
Latest_date= dt.datetime(2020,1,1)

In [107]:
 #RFM Code
RFMScores = DataFrame.groupby(['customer_id']).agg({'invoice_date':lambda x:(Latest_date-x.max()).days,'invoice_id':lambda x:len(x),'total_price':lambda x:x.sum()})
RFMScores['invoice_date']=RFMScores['invoice_date'].astype(int)
RFMScores.rename(columns={'invoice_date':'Regency',
                         'invoice_id':'Frequency',
                         'total_price':'Monetary'},inplace=True)
RFMScores.reset_index().head()

,customer_id,Regency,Frequency,Monetary
0,45,145,1,40887.0
1,50,36,1,1482.0
2,74,84,2,39588.0
3,86,11,1,3576.0
4,138,88,1,3960.0
